In [3]:
import tensorflow as tf
import numpy as np

In [4]:
tf.__version__

'2.1.0'

In [5]:
# first, we have 4 users and 5 movies in a matrix

user_movie = tf.constant([
    [4,  6, 8,  0, 0],
    [0,  0, 10, 0, 8],
    [0,  6, 0,  0, 3],
    [10, 9, 0,  5, 0]
])


# we have a movie to genre features tag matrix

movie_feats = tf.constant([
    [1,1,0,0,1],
    [1,1,0,0,0],
    [0,0,1,1,0],
    [1,0,1,1,0],
    [0,0,0,0,1]
])

In [11]:
# take each vector from the user row, turn it vertically, and multiply it by the row vectors of the movie-genre
weighted_feature_m = []
for i in range(len(user_movie)):
    weighted_feature_m.append(tf.expand_dims(user_movie[i], axis=1) * movie_feats)

In [12]:
weighted_feature_m

[<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[4, 4, 0, 0, 4],
        [6, 6, 0, 0, 0],
        [0, 0, 8, 8, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])>,
 <tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0, 10, 10,  0],
        [ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  8]])>,
 <tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0],
        [6, 6, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 3]])>,
 <tf.Tensor: shape=(5, 5), dtype=int32, numpy=
 array([[10, 10,  0,  0, 10],
        [ 9,  9,  0,  0,  0],
        [ 0,  0,  0,  0,  0],
        [ 5,  0,  5,  5,  0],
        [ 0,  0,  0,  0,  0]])>]

In [20]:
scaled_feature_m = []
for m in weighted_feature_m:
    agg_vec = tf.reduce_sum(m, axis=0)
    sum_agg_vec = tf.reduce_sum(agg_vec)
    scaled_feature_m.append(agg_vec / sum_agg_vec)

In [23]:
scaled_feature_m

[<tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.25, 0.25, 0.2 , 0.2 , 0.1 ])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.        , 0.        , 0.35714286, 0.35714286, 0.28571429])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.4, 0.4, 0. , 0. , 0.2])>,
 <tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.38095238, 0.3015873 , 0.07936508, 0.07936508, 0.15873016])>]

In [27]:
scaled_feature_m[0]

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([0.25, 0.25, 0.2 , 0.2 , 0.1 ])>

In [28]:
tf.tensordot

<function tensorflow.python.ops.math_ops.tensordot(a, b, axes, name=None)>

In [41]:
movie_feats

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[1, 1, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 1, 1, 0],
       [1, 0, 1, 1, 0],
       [0, 0, 0, 0, 1]])>

In [60]:
tf.matmul(tf.cast(movie_feats, dtype=tf.float64), tf.expand_dims(scaled_feature_m[0], axis=1))

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[0.6 ],
       [0.5 ],
       [0.4 ],
       [0.65],
       [0.1 ]])>

In [62]:
user_ratings = []
for i in range(len(user_movie)):
    user_ratings.append(tf.matmul(tf.cast(movie_feats, dtype=tf.float64), tf.expand_dims(scaled_feature_m[i], axis=1)))

In [74]:
user_ratings = (tf.stack(user_ratings, axis=0))

In [81]:
user_ratings_reshaped = tf.stack([tf.transpose(user_ratings[i]) for i in range(len(user_ratings))])

In [84]:
user_ratings_reshaped

<tf.Tensor: shape=(4, 1, 5), dtype=float64, numpy=
array([[[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ]],

       [[0.28571429, 0.        , 0.71428571, 0.71428571, 0.28571429]],

       [[1.        , 0.8       , 0.        , 0.4       , 0.2       ]],

       [[0.84126984, 0.68253968, 0.15873016, 0.53968254, 0.15873016]]])>

In [90]:
# our user has already seen some of the movie, so we need to mask those movies

tf.where(user_movie == 0)

<tf.Tensor: shape=(10, 2), dtype=int64, numpy=
array([[0, 3],
       [0, 4],
       [1, 0],
       [1, 1],
       [1, 3],
       [2, 0],
       [2, 2],
       [2, 3],
       [3, 2],
       [3, 4]], dtype=int64)>

In [104]:
user_ratings_reshaped.shape

TensorShape([4, 1, 5])

In [105]:
user_movie.shape

TensorShape([4, 5])

In [106]:
tf.reshape(user_ratings_reshaped, (4,5))

<tf.Tensor: shape=(4, 5), dtype=float64, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ],
       [0.28571429, 0.        , 0.71428571, 0.71428571, 0.28571429],
       [1.        , 0.8       , 0.        , 0.4       , 0.2       ],
       [0.84126984, 0.68253968, 0.15873016, 0.53968254, 0.15873016]])>

In [109]:
tf.where(user_movie == 0, tf.reshape(user_ratings_reshaped, (4,5)), tf.cast(user_movie, dtype=tf.float64))

<tf.Tensor: shape=(4, 5), dtype=float64, numpy=
array([[ 4.        ,  6.        ,  8.        ,  0.65      ,  0.1       ],
       [ 0.28571429,  0.        , 10.        ,  0.71428571,  8.        ],
       [ 1.        ,  6.        ,  0.        ,  0.4       ,  3.        ],
       [10.        ,  9.        ,  0.15873016,  5.        ,  0.15873016]])>

In [110]:
user_movie

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[ 4,  6,  8,  0,  0],
       [ 0,  0, 10,  0,  8],
       [ 0,  6,  0,  0,  3],
       [10,  9,  0,  5,  0]])>